In [1]:
import numpy as np
from scipy.optimize import linprog
from ipynb.fs.full.functions import nn2na, get_selected_arcs, arc_usage, get_min_cut

In [2]:
#Import data
NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [1, 0, 0, 0, 0, 0]])
beq = np.array([0, 0, 0, 0, 0, 0])
C = np.array([0, 0, 0, 0, 0, 0, 0, -1])
upper_b = [7, 1, 2, 3, 2, 1, 2, None]

In [3]:
# DATA transforming for optimization:
Aeq, arc_idxs = nn2na(NN)
bounds = tuple([(0, upper_b[arcs]) for arcs in range(0, Aeq.shape[1])])

In [4]:
print(bounds)

((0, 7), (0, 1), (0, 2), (0, 3), (0, 2), (0, 1), (0, 2), (0, None))


In [5]:
print('## Optimizer inputs ## \n'
      'Cost vector: %s \n'
      'A_eq Node-Arc matrix:\n%s \n'
      'b_eq demand-supply vector: %s \n'
      'Bounds of each X arc variable: %s \n' % (C, Aeq, beq, bounds)) 

## Optimizer inputs ## 
Cost vector: [ 0  0  0  0  0  0  0 -1] 
A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0  0 -1]
 [-1  0  1  1  0  0  0  0]
 [ 0 -1  0  0  1  0  0  0]
 [ 0  0 -1  0  0  1  0  0]
 [ 0  0  0  0 -1  0  1  0]
 [ 0  0  0 -1  0 -1 -1  1]] 
b_eq demand-supply vector: [0 0 0 0 0 0] 
Bounds of each X arc variable: ((0, 7), (0, 1), (0, 2), (0, 3), (0, 2), (0, 1), (0, 2), (0, None)) 



In [6]:
# OPTIMIZE:
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')
res.message

C:\Users\joaquin\Anaconda3\lib\site-packages\scipy\optimize\_linprog_util.py:704: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  warn(redundancy_warning, OptimizeWarning)


'Optimization terminated successfully.'

In [7]:
# Results equations
arc_use = arc_usage(arc_idxs, res.x.astype(int))
result = res.fun*(-1)
mincut = get_min_cut(arc_idxs, res.x, np.array(upper_b))

In [10]:
print('### Results of Max Flow ### \n'
      'The max flow circulating across the system is: %s \n'
      'The flow circulating through each arc is: %s \n'
      'The arcs that form the minimum cut are: %s'%(result, arc_use, mincut))

### Results of Max Flow ### 
The max flow circulating across the system is: 5.0 
The flow circulating through each arc is: {(0, 1): 4, (0, 2): 1, (1, 3): 1, (1, 5): 3, (2, 4): 1, (3, 5): 1, (4, 5): 1, (5, 0): 5} 
The arcs that form the minimum cut are: [(0, 2), (1, 5), (3, 5)]
